In [ ]:
%%capture
import operator

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import importlib as imp

from collections import namedtuple
from random import sample, shuffle
from functools import reduce
from itertools import accumulate
from math import floor, ceil, sqrt, log, pi
from matplotlib import pyplot as plt
from tensorflow.keras import layers, utils, losses, models as mds, optimizers

if imp.util.find_spec('aggdraw'): import aggdraw
if imp.util.find_spec('tensorflow_addons'): from tensorflow_addons import layers as tfa_layers
if imp.util.find_spec('tensorflow_models'): from official.vision.beta.ops import augment as visaugment
if imp.util.find_spec('tensorflow_probability'): from tensorflow_probability import distributions as tfd
if imp.util.find_spec('keras_tuner'): import keras_tuner as kt

In [ ]:
# Dataset image size
IMG_SIZE = 264
N_CLASSES = 102

def preprocess(image, *args):
    image = tf.image.resize_with_pad(image, IMG_SIZE, IMG_SIZE)
    image /= 255
    return (image, *args)

train_ds, val_ds = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation'],
    as_supervised=True,
    read_config=tfds.ReadConfig(try_autocache=False)
)

train_ds = train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def create_model(lr=0.001):
    m = tf.keras.Sequential([
        layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        layers.MaxPool2D(2),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPool2D(2),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPool2D(2),
        layers.Flatten(),
        layers.Dense(N_CLASSES, activation='softmax')
    ])
    
    optimizer = tf.keras.optimizers.Nadam(lr)
    m.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics='accuracy')

    return m

def create_with_params(lr):
    return create_model(lr)

In [ ]:
def build_model_with_hp():
    def build_model(hp):
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        bs = hp.Choice('bs', [8, 16, 24, 32])
        m = create_with_params(lr=lr)

        return m
    
    return build_model

class RandomSearchWithBatch(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        augs = tf.keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
        ])
        print(trial.hyperparameters.values)
        train_ds = args[0]
        val_ds = kwargs.get('validation_data')
        batch_size = trial.hyperparameters['bs']

        tds = train_ds.shuffle(batch_size*20, reshuffle_each_iteration=True).batch(batch_size)
        tds = tds.map(lambda x,y: (augs(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE).take(1)
        vds = val_ds.batch(batch_size).cache().take(1) if val_ds else None

        # Update Trial Arguments
        args = tds, *args[1:]
        kwargs['validation_data'] = vds

        return super(RandomSearchWithBatch, self).run_trial(trial, *args, **kwargs)

# Tune Hyper Parameters
tuner = RandomSearchWithBatch(build_model_with_hp(), objective='val_loss', max_trials=7)
tuner.search(train_ds, epochs=10, validation_data=val_ds)

# Select best parameters
model = tuner.get_best_models()[0]
BATCH_SIZE = tuner.get_best_hyperparameters()[0]['bs']
tuner.results_summary()
tuner.get_best_hyperparameters()[0].values